# Validação Cruzada em Machine Learning

## Objetivos

- Compreender a importância da validação cruzada
- Implementar diferentes tipos de validação cruzada
- Avaliar a estabilidade de modelos
- Evitar overfitting através de validação adequada

## Pré-requisitos

- Conceitos básicos de ML
- Divisão treino/teste
- Métricas de avaliação


## 1. Por que Validação Cruzada?

A validação cruzada é uma técnica fundamental para:

- **Estimar performance real** do modelo
- **Detectar overfitting** de forma mais robusta
- **Comparar modelos** de forma justa
- **Usar dados limitados** de forma eficiente


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, TimeSeriesSplit, train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
import warnings

warnings.filterwarnings("ignore")

## 2. Problema da Divisão Simples Treino/Teste


In [ ]:
# Criar dataset de exemplo
X, y = make_classification(n_samples=1000, n_features=20, n_informative=10, n_redundant=10, random_state=42)

# Simular múltiplas divisões treino/teste
scores = []
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=i)

    model = LogisticRegression(random_state=42)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    scores.append(score)

print(f"Accuracy média: {np.mean(scores):.3f}")
print(f"Desvio padrão: {np.std(scores):.3f}")
print(f"Variação: {np.min(scores):.3f} - {np.max(scores):.3f}")

# Visualizar distribuição
plt.figure(figsize=(10, 6))
plt.hist(scores, bins=20, alpha=0.7, edgecolor="black")
plt.axvline(np.mean(scores), color="red", linestyle="--", label=f"Média: {np.mean(scores):.3f}")
plt.xlabel("Accuracy")
plt.ylabel("Frequência")
plt.title("Variabilidade da Performance com Diferentes Divisões Treino/Teste")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 3. K-Fold Cross Validation

O K-Fold divide os dados em K partes (folds) e usa cada parte como teste uma vez.


In [ ]:
# Implementação manual do K-Fold
def manual_kfold_cv(X, y, model, k=5):
    n_samples = len(X)
    fold_size = n_samples // k
    scores = []

    for i in range(k):
        # Definir índices do fold de teste
        start_idx = i * fold_size
        if i == k - 1:  # Último fold pega o resto
            end_idx = n_samples
        else:
            end_idx = (i + 1) * fold_size

        # Dividir dados
        test_indices = range(start_idx, end_idx)
        train_indices = list(range(0, start_idx)) + list(range(end_idx, n_samples))

        X_train_fold = X[train_indices]
        X_test_fold = X[test_indices]
        y_train_fold = y[train_indices]
        y_test_fold = y[test_indices]

        # Treinar e avaliar
        model_copy = LogisticRegression(random_state=42)
        model_copy.fit(X_train_fold, y_train_fold)
        score = model_copy.score(X_test_fold, y_test_fold)
        scores.append(score)

        print(f"Fold {i+1}: Accuracy = {score:.3f}")

    return scores


# Testar implementação manual
print("=== Implementação Manual K-Fold ===")
manual_scores = manual_kfold_cv(X, y, LogisticRegression(), k=5)
print(f"\nMédia: {np.mean(manual_scores):.3f} ± {np.std(manual_scores):.3f}")

In [ ]:
# Usar sklearn para K-Fold
print("\n=== Sklearn K-Fold ===")
model = LogisticRegression(random_state=42)
cv_scores = cross_val_score(model, X, y, cv=5, scoring="accuracy")

for i, score in enumerate(cv_scores):
    print(f"Fold {i+1}: Accuracy = {score:.3f}")

print(f"\nMédia: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")

## 4. Stratified K-Fold

Mantém a proporção das classes em cada fold - essencial para datasets desbalanceados.


In [ ]:
# Criar dataset desbalanceado
X_imb, y_imb = make_classification(n_samples=1000, n_features=20, weights=[0.9, 0.1], random_state=42)

print("Distribuição das classes:")
unique, counts = np.unique(y_imb, return_counts=True)
for cls, count in zip(unique, counts):
    print(f"Classe {cls}: {count} ({count/len(y_imb)*100:.1f}%)")

# Comparar K-Fold regular vs Stratified
print("\n=== K-Fold Regular ===")
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
for i, (train_idx, test_idx) in enumerate(kfold.split(X_imb)):
    y_train_fold = y_imb[train_idx]
    y_test_fold = y_imb[test_idx]

    train_dist = np.bincount(y_train_fold) / len(y_train_fold)
    test_dist = np.bincount(y_test_fold) / len(y_test_fold)

    print(
        f"Fold {i+1} - Treino: [{train_dist[0]:.2f}, {train_dist[1]:.2f}] "
        f"Teste: [{test_dist[0]:.2f}, {test_dist[1]:.2f}]"
    )

print("\n=== Stratified K-Fold ===")
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for i, (train_idx, test_idx) in enumerate(skfold.split(X_imb, y_imb)):
    y_train_fold = y_imb[train_idx]
    y_test_fold = y_imb[test_idx]

    train_dist = np.bincount(y_train_fold) / len(y_train_fold)
    test_dist = np.bincount(y_test_fold) / len(y_test_fold)

    print(
        f"Fold {i+1} - Treino: [{train_dist[0]:.2f}, {train_dist[1]:.2f}] "
        f"Teste: [{test_dist[0]:.2f}, {test_dist[1]:.2f}]"
    )

## 5. Time Series Split

Para dados temporais, não podemos misturar passado e futuro!


In [ ]:
# Simular dados de série temporal
np.random.seed(42)
n_samples = 1000
time = np.arange(n_samples)
trend = 0.01 * time
seasonal = 2 * np.sin(2 * np.pi * time / 50)
noise = np.random.normal(0, 0.5, n_samples)
y_ts = trend + seasonal + noise


# Criar features baseadas em lag
def create_lag_features(y, n_lags=5):
    X = np.zeros((len(y) - n_lags, n_lags))
    for i in range(n_lags):
        X[:, i] = y[i : len(y) - n_lags + i]
    return X


X_ts = create_lag_features(y_ts, n_lags=5)
y_ts_target = y_ts[5:]  # Target é o próximo valor

print(f"Shape dos dados temporais: X={X_ts.shape}, y={y_ts_target.shape}")

# Visualizar Time Series Split
tscv = TimeSeriesSplit(n_splits=5)

plt.figure(figsize=(15, 8))
for i, (train_idx, test_idx) in enumerate(tscv.split(X_ts)):
    plt.subplot(2, 3, i + 1)
    plt.plot(train_idx, y_ts_target[train_idx], "b-", label="Treino", alpha=0.7)
    plt.plot(test_idx, y_ts_target[test_idx], "r-", label="Teste", alpha=0.7)
    plt.title(f"Split {i+1}")
    plt.legend()
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Avaliar com Time Series CV
model_ts = LinearRegression()
cv_scores_ts = cross_val_score(
    model_ts, X_ts, y_ts_target, cv=TimeSeriesSplit(n_splits=5), scoring="neg_mean_squared_error"
)

print(f"\nTime Series CV - MSE: {-cv_scores_ts.mean():.3f} ± {cv_scores_ts.std():.3f}")

## 6. Comparando Modelos com Cross Validation


In [ ]:
# Comparar diferentes modelos
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

models = {
    "Logistic Regression": LogisticRegression(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(random_state=42),
    "Naive Bayes": GaussianNB(),
}

# Avaliar cada modelo com CV
results = {}
for name, model in models.items():
    cv_scores = cross_val_score(model, X, y, cv=5, scoring="accuracy")
    results[name] = cv_scores
    print(f"{name:20}: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")

# Visualizar comparação
plt.figure(figsize=(12, 6))
plt.boxplot(results.values(), labels=results.keys())
plt.ylabel("Accuracy")
plt.title("Comparação de Modelos com Cross Validation")
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Nested Cross Validation

Para seleção de hiperparâmetros sem vazamento de dados.


In [ ]:
from sklearn.model_selection import GridSearchCV


# Nested CV para Random Forest
def nested_cross_validation(X, y, outer_cv=5, inner_cv=3):
    # Parâmetros para busca
    param_grid = {"n_estimators": [50, 100, 200], "max_depth": [3, 5, 7, None]}

    # CV externo
    outer_scores = []
    kfold_outer = KFold(n_splits=outer_cv, shuffle=True, random_state=42)

    for fold, (train_idx, test_idx) in enumerate(kfold_outer.split(X)):
        X_train_outer, X_test_outer = X[train_idx], X[test_idx]
        y_train_outer, y_test_outer = y[train_idx], y[test_idx]

        # CV interno para seleção de hiperparâmetros
        rf = RandomForestClassifier(random_state=42)
        grid_search = GridSearchCV(rf, param_grid, cv=inner_cv, scoring="accuracy", n_jobs=-1)

        # Encontrar melhores parâmetros no conjunto de treino
        grid_search.fit(X_train_outer, y_train_outer)

        # Avaliar no conjunto de teste
        best_model = grid_search.best_estimator_
        score = best_model.score(X_test_outer, y_test_outer)
        outer_scores.append(score)

        print(f"Fold {fold+1}: Score = {score:.3f}, " f"Best params = {grid_search.best_params_}")

    return outer_scores


print("=== Nested Cross Validation ===")
nested_scores = nested_cross_validation(X, y)
print(f"\nPerformance estimada: {np.mean(nested_scores):.3f} ± {np.std(nested_scores):.3f}")

## 8. Resumo e Boas Práticas

### Escolhendo o Tipo de CV:

- **K-Fold**: Dados gerais, balanceados
- **Stratified K-Fold**: Classificação com classes desbalanceadas
- **Time Series Split**: Dados temporais
- **Leave-One-Out**: Datasets muito pequenos

### Parâmetros Importantes:

- **K=5 ou K=10**: Padrão para a maioria dos casos
- **shuffle=True**: Importante para dados ordenados
- **random_state**: Para reprodutibilidade

### Cuidados:

- Nested CV para seleção de hiperparâmetros
- Mesmo pré-processamento em todos os folds
- Não vazar informação entre treino e teste
